In [42]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Load Data

In [43]:
# Load 2017 trips csv
df = pd.read_csv('metro-bike-share-trips-2017-q4-v2.csv', usecols=[2, 3, 4, 7])
df

,start_time,end_time,start_station,end_station
0,2017-12-30 10:20:00,2017-12-30 10:22:00,3000,3000
1,2017-12-29 08:03:00,2018-01-01 09:27:00,3022,3000
2,2017-12-30 23:44:00,2018-01-01 18:24:00,3007,3000
3,2017-12-31 12:43:00,2018-01-01 19:26:00,3047,3000
4,2017-12-30 22:13:00,2018-01-02 12:20:00,3023,3000
5,2017-12-31 20:40:00,2018-01-03 12:39:00,3005,3000
6,2017-12-29 11:29:00,2017-12-29 14:50:00,4211,3000
7,2017-12-30 10:24:00,2017-12-30 10:25:00,3000,3000
8,2017-12-30 10:20:00,2017-12-30 10:22:00,3000,3000
9,2017-12-30 00:18:00,2017-12-30 11:48:00,3024,3000


In [45]:
# Load bike id mapping csv
bike_ids = pd.read_csv('metro-bike-share-stations-2019-01-07.csv')

# Feature Extraction

In [46]:
from datetime import datetime

def convert_to_seconds(time):
    """
    Converts a YYYY-MM-DD HH:MM:SS timestamp to Unix time (in seconds).
    """
    utc_time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    epoch_time = (utc_time - datetime(1970, 1, 1)).total_seconds()
    return epoch_time

# Random Forest

In [47]:
def grid_search(X_train, y_train):
    """
    Perform grid search for hyperparameters.
    """
    reg = RandomForestClassifier()
    param_grid = {
            "n_estimators"      : [10,50,100,500],
            "max_features"      : ["auto", "sqrt", "log2"],
            "min_samples_leaf" : [1,5,10,20]
            }
    grid = GridSearchCV(reg, param_grid, verbose=0)

    grid.fit(X_train, y_train)

    return grid.best_score_ , grid.best_params_

In [48]:
def create_rand_forest(X_train, y_train):
    
    # determine parameters
#     best_score, best_params = grid_search(X_train, y_train)
#     mf = best_params['max_features']
#     msl = best_params['min_samples_leaf']
#     ne = best_params['n_estimators']
    
    rfr = RandomForestClassifier(n_estimators=10)
    rfr.fit(X_train, y_train)
    
    return rfr

In [49]:
x_cols = ['start_time', 'end_time', 'start_station']
x = df[x_cols]
x['start_time'] = df['start_time'].apply(convert_to_seconds)
x['end_time'] = df['end_time'].apply(convert_to_seconds)

x.head()

C:\Users\Swathi\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Swathi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,start_time,end_time,start_station
0,1.514629e+09,1.514629e+09,3000
1,1.514535e+09,1.514799e+09,3022
2,1.514677e+09,1.514831e+09,3007
3,1.514724e+09,1.514835e+09,3047
4,1.514672e+09,1.514896e+09,3023


In [50]:
y = df['end_station']
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.20, random_state=42)

In [51]:
rfr = create_rand_forest(x_train, y_train)

In [55]:
# Get predictions
train_pred = rfr.predict(x_train)
val_pred = rfr.predict(x_val)
n, d = x_val.shape
data = np.random.rand(n, 1)
f = lambda x: 1 if x > 0.5 else -1
base_pred = np.array(list(map(f, data)))

# calculate error

train_err = (y_train - train_pred)
val_err = (y_val - val_pred)
base_err = (base_pred - val_pred)

print('Training Error:', np.count_nonzero(train_err)/len(train_err))
print('Validation Error:', np.count_nonzero(val_err)/len(val_err))
print('Baseline Error:', np.count_nonzero(base_err)/len(base_err))

train_pred

Training Error: 0.02957645117691457
Validation Error: 0.6819490275925015
Baseline Error: 1.0


array([4214, 3046, 3082, ..., 4165, 3000, 3030], dtype=int64)